## Runnning Base Model

In [1]:
from llama_cpp import Llama

# Download the model file first to your local directory
model_path = "./Models/finance-chat.Q3_K_M.gguf"  # Example GGUF file

# Initialize the Llama model with the GGUF file
llm = Llama(
    model_path=model_path,
    n_ctx=4096,  # The max sequence length to use
    n_threads=8,  # The number of CPU threads to use
    n_gpu_layers=35  # The number of layers to offload to GPU
)

# Example usage of the model
output = llm(
    "[INST] <<SYS>>\n{system_message}\n<</SYS>>\n{prompt} [/INST]",  # The prompt
    max_tokens=512,  # Maximum number of tokens to generate
    stop=["</s>"],  # Stop token
    echo=True  # Whether to echo the prompt in the output
)

# Print the generated output
print(output)


ModuleNotFoundError: No module named 'llama_cpp'

## Training the model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

# Load your dataset
data_path = "your_dataset.csv"
df = pd.read_csv(data_path)

# Preprocess data
# Assuming 'prompt' is your input and 'output' is the target
dataset = Dataset.from_pandas(df)
def preprocess(examples):
    return tokenizer(examples["prompt"], max_length=512, truncation=True, padding="max_length"), tokenizer(examples["output"], max_length=512, truncation=True, padding="max_length")
dataset = dataset.map(preprocess, batched=True)

# Load tokenizer and model
model_name = "your_pretrained_model_name_here"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./model_output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Train
trainer.train()

# Save the model
trainer.save_model("./fine_tuned_model")


## Testing the Model